# Setup and Disclaimer

Run the following cell to install all the necessary.

Disclaimer, parts of this tutorial are highly inspired by a [Hugging Face course](https://huggingface.co/learn/nlp-course/chapter1/1)

We will use PyTorch as backbone

In [1]:
! pip install transformers datasets peft evaluate "transformers[torch]"

# Questions

### 1 Which metrics or patterns might help to detect AI-generated text?


### 2 What is the minimum amount of RAM I would need to run a batch of 4 samples through a GPT-3 instance? (Hint: think about the no. of weights)


### 3 You want to build a classification architecture on the [AG News dataset](https://huggingface.co/datasets/ag_news). Describe how you would use BERT to build a classification architecture?


### 4 How many output neurons would your last layer have? What activation function would you use there?


### 5 Suppose now we want to use GPT-3 instead and want to do zero-shot learning. Consider the input sentence $x_0=$ _"The Social Computing Group at TUM has just released GPT-5, that is impressive!"_  labelled as $y_0=$_"Sci/Tech"_. Design a reasonable prompt for $(x_0,y_0)$.


### 6 Now add demonstrations to your prompt to perform in-context learning. Are you still performing zero-shot learning? If not, what instead? Explicitely state which is the pattern $f$ and which is the verbalizer $v$.


### 7 Suppose we are now given additional data to fine-tune your model, but retraining 175B parameters is absolutely unfeasible. How can parameter-efficient-tuning help us?


### 8 Let's suppose you pick LoRA as efficient method for fine-tuning but you don't want to add ANY additional parameter once deployed. What can you do?


### 9 Why was RL (initially) the only viable option for intergrating human preferences into LLMs? What does it mean that "the environment is not differentiable"?


# Pipeline

`pipeline()` is the simplest way to use a Hugging Face model. **Model** is an umbrella term describing an **architecture** (i.e. the skeleton/structure of the model) and a **checkpoint** (i.e. the set of weights that we load on it).

For example, BERT is an architecture while `bert-base-cased`, a set of weights trained by the Google team for the first release of BERT, is a checkpoint. However, one can say “the BERT model” and “the `bert-base-cased model'”

In [2]:
from transformers import pipeline

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
classifier = pipeline("sentiment-analysis", checkpoint)
classifier(["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"])

/home/artem/programming/nlp_lecture_practice/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/artem/programming/nlp_lecture_practice/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [4]:
generator = pipeline("text-generation")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to implement a number of advanced techniques with Java, Java SE, OpenJDK, Scala and Scala.'},
 {'generated_text': 'In this course, we will teach you how to use SQL Server 2.\n\nThe first step is to create an account. To do this,'}]

As you can see, if you provide no model checkpoint, Hugging Face will automatically pick an adequate one.

Check [here](https://huggingface.co/learn/nlp-course/chapter1/3?fw=pt) other usage examples for the pipeline function

Now use pipeline to perform a different task

In [6]:
### YOUR CODE HERE ###

checkpoint = "bert-base-uncased"
unmasker = pipeline("fill-mask", checkpoint)
unmasker("This course will teach you all about [MASK] models.", top_k=2)

######################

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.33661195635795593,
  'token': 2535,
  'token_str': 'role',
  'sequence': 'this course will teach you all about role models.'},
 {'score': 0.08371765166521072,
  'token': 2449,
  'token_str': 'business',
  'sequence': 'this course will teach you all about business models.'}]

# Behind the Pipeline

The Pipeline operator takes care of all steps: tokenization, going through the model, and post-processing. We now go one level lower to gain a better understanding of how everything works.

### Tokenizer

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

sequence = "I've been waiting for a HuggingFace course my whole life."

print(tokenizer(sequence))

tokens = tokenizer.tokenize(sequence) # You can also use the __call__ function directly

print(tokens)

ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

decoded_string = tokenizer.decode(ids)

print(decoded_string)

{'input_ids': [101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.']
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]
i've been waiting for a huggingface course my whole life.


In [7]:
tokens_test = [101, 102]
decoded_string = tokenizer.decode(tokens_test)
decoded_string

'[CLS] [SEP]'

What are the tokens with indices 101 and 102? Find out!

[CLS] and [SEP].

### Padding
To put inputs together is called batching. In order to batch sequences together, they need to be of the same length, which implies you need **pad** the inputs to ensure this requirement is met.

In [8]:
from transformers import AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# This will work

batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(batched_ids)).logits)

# But this this will fail
batched_ids = [
    [200, 200, 200],
    [200, 200]
]

print(model(torch.tensor(batched_ids)).logits)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[-0.0924, -0.0277],
        [-0.0907, -0.0168]], grad_fn=<AddmmBackward0>)


ValueError: expected sequence of length 3 at dim 1 (got 2)

### Attention Mask
Remember that the model attends to all tokens, and we don't want our predictions to be depended on the padding tokens. We use **attention masks** to indicate which tokens should be attended and which not.

In [9]:
# This prediction
sentence_1_ids = [[200, 200, 200]]
sentence_2_ids = [[200, 200]]

# will be different from this one
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

# so we add the attention masks
attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

print(f"{model(torch.tensor(sentence_1_ids)).logits}\n"\
      f"{model(torch.tensor(sentence_2_ids)).logits}")
print(model(torch.tensor(batched_ids)).logits)
outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[-0.0924, -0.0277]], grad_fn=<AddmmBackward0>)
tensor([[-0.0024,  0.1289]], grad_fn=<AddmmBackward0>)
tensor([[-0.0924, -0.0277],
        [-0.0907, -0.0168]], grad_fn=<AddmmBackward0>)
tensor([[-0.0924, -0.0277],
        [-0.0024,  0.1289]], grad_fn=<AddmmBackward0>)


### Truncate
Most transformers handle sequences of up to 512 or 1024 tokens, and will crash when asked to process longer sequences. There are two solutions to this problem: either you (1) use a model with a longer supported sequence length or (2)truncate your sequences.

In [10]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Will pad the sequences up to the maximum sequence length
model_inputs = tokenizer(sequences, padding="longest")

# Will pad the sequences up to the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")

# Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)

# Will truncate the sequences that are longer than the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)

# Will truncate the sequences that are longer than the specified max length
model_inputs = tokenizer(sequences, max_length=8, truncation=True)

### Through the Model

Why does the following code fail? Can you fix it?

Models by default handle multiple sentences, so we need to add a dimension to our `input_ids`

In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

# The issue here is that the model expects the input tensors to have an additional dimension for batch size.
# We can fix this by adding an extra dimension using unsqueeze() method.

tokens = tokenizer.tokenize(sequence, add_special_tokens=True)
print(tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)
attention_mask = [0] + [1 for _ in range(len(ids)-2)] + [0]

input_ids = torch.tensor(ids).unsqueeze(0)  # Add an extra dimension for batch size
attention_mask = torch.tensor(attention_mask).unsqueeze(0)  # Add an extra dimension for batch size

print(len(attention_mask[0]), len(input_ids[0]))

print(input_ids)
print(attention_mask)

# Now this line will not fail.
model(input_ids, attention_mask=attention_mask).logits


['[CLS]', 'i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.', '[SEP]']
[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
16 16
tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])
tensor([[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])


tensor([[-1.4796,  1.6475]], grad_fn=<AddmmBackward0>)

Now let's apply everything we have learned and let's run the model on `raw_inputs` without using the `pipeline()` function. We placed some importa as hints. We're using the same inputs and model checkpoint, so you should get the same output as before!

In [57]:
from transformers import AutoModelForSequenceClassification
from torch import argmax
from torch.nn.functional import softmax

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]

### YOUR CODE HERE ###

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

tokens = []
for i in raw_inputs:
    tokens.append(tokenizer.tokenize(i, add_special_tokens=True))

#print(tokens)
max_tokens = len(max(tokens, key=len))
#print(max_tokens)
print(tokens)
class_res = []
for i in tokens:
    ids = tokenizer.convert_tokens_to_ids(i)
    ids.extend([tokenizer.pad_token_id] * (max_tokens-len(i)))
    att_mask = [1 if token != tokenizer.pad_token_id else 0 for token in i]
    input_ids = torch.tensor(ids).unsqueeze(0)
    res = model(input_ids, attention_mask=attention_mask).logits
    class_res.append(max(softmax(res)))

print(class_res)

######################

[['[CLS]', 'i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.', '[SEP]'], ['[CLS]', 'i', 'hate', 'this', 'so', 'much', '!', '[SEP]']]
[tensor([0.0420, 0.9580], grad_fn=<UnbindBackward0>), tensor([0.9982, 0.0018], grad_fn=<UnbindBackward0>)]


/tmp/ipykernel_10402/3426070204.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  class_res.append(max(softmax(res)))


Very likely your code does not work if you use the class `Automodel` instead of `AutoModelForSequenceClassification`. Why not?



# Training

We start by loading a dataset with the dataset library

In [12]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
print(raw_datasets)

raw_train_dataset = raw_datasets['train']
print(raw_train_dataset.features.keys())

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})
dict_keys(['sentence1', 'sentence2', 'label', 'idx'])


Glue mrpc is a classification task: (sentence_1, sentence_2) -> Are they semantically equivalent?

By running the tokenizer we can observe the model expects the inputs to be of the form [CLS] sentence1 [SEP] sentence2 [SEP]. The token_type_ids determines which part is sentence_1 and which part is sentence_"

Do not worry if you don't see token_type_ids in your tokenized inputs: as long as you use the same checkpoint for the tokenizer and the model, everything will be fine as the tokenizer knows what to provide to its model.

In [13]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

inputs = tokenizer("This is the first sentence.", "This is the second one.")
print(inputs)

print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']


Simply applying the tokenizer to the dataset would require to store the entire dataset in memory. Instead, we would like to only store the current batch in memory by appling the `map()` function with argument `batched=True`.

This also enables to use dynamic padding through a `DataCollatorWithPadding`, i.e. we pad to the maximum length within the batch and not the entire dataset.

Define a `tokenize_function()` which we can then apply to the entire dataset. Don't forget, each sample is made of two sentences!

In [40]:
from transformers import DataCollatorWithPadding
import numpy as np

### YOUR CODE HERE ###

def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, padding='max_length')

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True) #batched_true is the whole point of doing this!

samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Print shapes for one batch
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

Map: 100%|██████████| 3668/3668 [00:00<00:00, 5959.37 examples/s]


{'input_ids': torch.Size([8, 512]),
 'token_type_ids': torch.Size([8, 512]),
 'attention_mask': torch.Size([8, 512]),
 'labels': torch.Size([8])}

In [29]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

Now that we can tokenize batches dynamically. We can sequentially feed them into the model to train it. We use the high-level `Trainer` class.

Please notice that `Trainer` doesn't provide us with evaluation metrics by default. We can retrieve them from the dataset through `evaluate` and compute them for our model.


In [16]:
import numpy as np
# import evaluate

# predictions = trainer.predict(tokenized_datasets["validation"])
# preds = np.argmax(predictions.predictions, axis=-1)

# metric = evaluate.load("glue", "mrpc")
# metric.compute(predictions=preds, references=predictions.label_ids)

Use `evaluate.load()` and `metric.compute()` to complete the `compute_metric()` function. We can then pass it to our `Trainer` object to track such metrics during training.

# Behind the Trainer

We go one level lower the Trainer class and implement the training procedure in PyTorch.

Starting from the same setup:

In [31]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

We setup the Pytorch DataLoaders

In [34]:
from torch.utils.data import DataLoader

tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
#print(tokenized_datasets)
tokenized_datasets.set_format("torch")
#print(tokenized_datasets["train"].column_names)
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
#print(train_dataloader)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

ValueError: Column name ['sentence1', 'idx', 'sentence2'] not in the dataset. Current columns in the dataset: ['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [38]:
for batch in train_dataloader:
    print(i)


ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.

We setup an Optimizer:

In [36]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


/home/artem/programming/nlp_lecture_practice/.venv/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Now you write the PyTorch code inside the training loop.

In [37]:
from tqdm.auto import tqdm
import torch
import numpy as np

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        print(batch)
        # batch = {k: v.to(device) for k, v in batch.items()}  # Move batch to device
        # outputs = model(**batch)  # Forward pass
        # loss = outputs.loss  # Compute loss
        # loss.backward()  # Backward pass
        # optimizer.step()  # Optimize
        # optimizer.zero_grad()  # Zero gradients
        # progress_bar.update(1)  # Update progress bar




  0%|          | 0/1377 [03:54<?, ?it/s]


ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.

Does it work? Congrats! You implement a training loop from scratch and you can now run the final evaluation. You should get similar results as with the `Trainer` class.

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

# Parameter-Efficient Fine-Tuning

You're given the same setup:

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification


raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

We provide you with these two functions for counting active parameters and defining evaluation metrics.

In [ ]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return {'Total': total_params, 'Trainable': trainable_params}

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Use LoRA by defining a configuration (`LoRAConfig` object) and use it with the trainer class to efficiently fine-tune our model

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np


### YOUR CODE HERE ###



######################


# Count parameters before LoRA
before_lora_count = count_parameters(model)
print(f"Before LoRA:\n{before_lora_count}")

# Apply LoRA to the model
lora_model = get_peft_model(model, lora_config)

# Count parameters after LoRA
after_loara_count = count_parameters(lora_model)
print(f"After LoRA:\n{after_loara_count}")

training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")  # Hyperparams for the training

### YOUR CODE HERE ###



######################

trainer.train()